### Your Personal Stylist

we will begin by writing a scrapper to scrape products and their product descripttion.

In [4]:
#importing the libraries
import pandas as pd 
from parsel import Selector
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException  # This line is important
import time
import random
import os
import re

In [13]:
# Initialize Chrome options
opts = Options()
opts.add_argument("--start-maximized")
opts.add_argument("--disable-notifications")

# Set up the Chrome WebDriver
driver = webdriver.Chrome(options=opts)

# URLs of product categories
urls = [
    # "https://www.myntra.com/men-sweatshirts",
    "https://www.myntra.com/men-tshirts",  # Added missing comma here
    "https://www.myntra.com/men-casual-shirts",
    "https://www.myntra.com/men-formal-shirts",
    "https://www.myntra.com/men-jeans",
    "https://www.myntra.com/men-casual-trousers",
    "https://www.myntra.com/men-formal-trousers",
    "https://www.myntra.com/mens-shorts",
    "https://www.myntra.com/men-trackpants",
    "https://www.myntra.com/tops",
    "https://www.myntra.com/myntra-fashion-store?f=Categories%3ATshirts%3A%3AGender%3Amen%20women%2Cwomen",
    "https://www.myntra.com/women-jeans",
    "https://www.myntra.com/women-trousers",
    "https://www.myntra.com/women-shorts-skirts"
]

product_types = [
    # "men-sweatshirts",
    "men-tshirts","men-casual-shirts", "men-formal-shirts", "men-jeans",
    "men-casual-trousers", "men-formal-trousers", "mens-shorts", "men-trackpants","women-tops",
    "women-tshirts","women-jeans","women-trousers","women-shorts-skirts"
]

# urls=[ "https://www.myntra.com/men-sweatshirts"]
# product_types=["men-sweatshirts"]
# DataFrame to store extracted data
# List to keep track of error links
error_links = []
# Loop over each product category URL
for single_url in urls:
    driver.get(single_url)
    time.sleep(5)  # Wait for the page to load

    # Extract all product buy links
    buy_links = [a.get_attribute('href') for a in driver.find_elements(By.CSS_SELECTOR, 'a[data-refreshpage="true"]') if 'buy' in a.get_attribute('href')]
    buy_links = list(set(buy_links))  # Remove duplicates and convert to list
 
    for link in buy_links:
        try:
            driver.get(link)
            time.sleep(5)  # Wait for the page to load

            # Extract product details
            product_title = driver.find_element(By.CSS_SELECTOR, ".pdp-title").text
            product_name = driver.find_element(By.CSS_SELECTOR, ".pdp-name").text
            product_price = driver.find_element(By.CSS_SELECTOR, ".pdp-mrp").text
            product_price_disc = driver.find_element(By.CSS_SELECTOR, ".pdp-price").text
            product_details = driver.find_element(By.CSS_SELECTOR, ".pdp-product-description-content").text

            # Extract specifications
            specifications_elements = driver.find_elements(By.CSS_SELECTOR, ".index-row")
            specifications = {
                row.find_element(By.CSS_SELECTOR, ".index-rowKey").text: row.find_element(By.CSS_SELECTOR, ".index-rowValue").text
                for row in specifications_elements
            }

            # Extract reviews
            reviews_elements = driver.find_elements(By.CSS_SELECTOR, ".user-review-userReviewWrapper")
            reviews = []
            for review in reviews_elements:
                rating = review.find_element(By.CSS_SELECTOR, ".user-review-starRating").text
                review_text = review.find_element(By.CSS_SELECTOR, ".user-review-reviewTextWrapper").text
                reviewer_name = review.find_element(By.CSS_SELECTOR, ".user-review-left span:first-child").text
                review_date = review.find_element(By.CSS_SELECTOR, ".user-review-left span:last-child").text
                reviews.append({
                    "rating": rating,
                    "review_text": review_text,
                    "reviewer_name": reviewer_name,
                    "review_date": review_date
                })

            # Extract image URLs
            image_elements = driver.find_elements(By.CSS_SELECTOR, ".image-grid-image")
            image_urls = [img.get_attribute('style').split('url("')[1].split('")')[0] for img in image_elements]

            # Store the extracted data
            data = {
                "Product Title": product_title,
                "Product Name": product_name,
                "Product Price": product_price,
                "Product Discounted Price": product_price_disc,
                "Product Details": product_details,
                "Specifications": specifications,
                "Reviews": reviews,
                "Image URLs": image_urls,
                "Product Type": product_types[urls.index(single_url)],  # Match product type with URL
                "Buy Link": link  # Add the buy link to the data
            }

            # Append data to the DataFrame
            df = df.append(data, ignore_index=True)

        except Exception as e:
            error_links.append(link)
            print(f"Error processing link {link}: {e}")

# Save the DataFrame to a CSV file
df.to_csv("extracted_product_data.csv", index=False)

# Print error links, if any
if error_links:
    print("Error Links:")
    for error_link in error_links:
        print(error_link)

# Close the driver
driver.quit()

/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.appe

Error processing link https://www.myntra.com/track-pants/hrx+by+hrithik+roshan/hrx-by-hrithik-roshan-men-printed-detail-lifestyle-track-pants/21771948/buy: Message: no such element: Unable to locate element: {"method":"css selector","selector":".pdp-title"}
  (Session info: chrome=128.0.6613.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001010d3998 cxxbridge1$str$ptr + 1887096
1   chromedriver                        0x00000001010cbe00 cxxbridge1$str$ptr + 1855456
2   chromedriver                        0x0000000100cd0be0 cxxbridge1$string$len + 89508
3   chromedriver                        0x0000000100d14f10 cxxbridge1$string$len + 368852
4   chromedriver                        0x0000000100d4e4a4 cxxbridge1$string$len + 603752
5   chromedriver                        0x0000000100d09a08 cxxbridge1$string$len + 322508
6   chrom

/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.appe

Error processing link https://www.myntra.com/skirts/berrylush/berrylush-white-floral-vacay-attire/18637610/buy: Message: no such element: Unable to locate element: {"method":"css selector","selector":".pdp-title"}
  (Session info: chrome=128.0.6613.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x00000001010d3998 cxxbridge1$str$ptr + 1887096
1   chromedriver                        0x00000001010cbe00 cxxbridge1$str$ptr + 1855456
2   chromedriver                        0x0000000100cd0be0 cxxbridge1$string$len + 89508
3   chromedriver                        0x0000000100d14f10 cxxbridge1$string$len + 368852
4   chromedriver                        0x0000000100d4e4a4 cxxbridge1$string$len + 603752
5   chromedriver                        0x0000000100d09a08 cxxbridge1$string$len + 322508
6   chromedriver                        0x0000000100d

/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data, ignore_index=True)
/var/folders/18/zngs9hrs7b11mktl5btn5_h80000gn/T/ipykernel_5727/526609069.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.appe

Error Links:
https://www.myntra.com/track-pants/hrx+by+hrithik+roshan/hrx-by-hrithik-roshan-men-printed-detail-lifestyle-track-pants/21771948/buy
https://www.myntra.com/skirts/berrylush/berrylush-white-floral-vacay-attire/18637610/buy


In [14]:
df.to_excel("output.xlsx")

In [15]:
df

,Product Title,Product Name,Product Price,Product Discounted Price,Product Details,Specifications,Reviews,Image URLs,Product Type,Buy Link
0,H&M,Relaxed Fit Sweatshirt,MRP,MRP₹ 799,Top in sweatshirt fabric made from a cotton bl...,"{'Sleeve Length': 'Long Sleeves', 'Neck': 'Rou...","[{'rating': '5', 'review_text': 'That's exactl...","[https://assets.myntassets.com/h_720,q_90,w_54...",men-sweatshirts,https://www.myntra.com/sweatshirts/h%26m/hm-re...
1,Roadster,Men Charcoal Solid Hooded Sweatshirt,MRP ₹999,₹599,"Charcoal solid sweatshirt, has a hood, long sl...","{'Sleeve Length': 'Long Sleeves', 'Neck': 'Hoo...","[{'rating': '5', 'review_text': 'I have been u...","[https://assets.myntassets.com/h_720,q_90,w_54...",men-sweatshirts,https://www.myntra.com/sweatshirts/roadster/ro...
2,Quechua By Decathlon,Men Orange Solid Hiking and Trekking Sweatshirt,MRP ₹899,₹699,Orange sweathirt for men\nPattern: solid\nRegu...,"{'Sleeve Length': 'Long Sleeves', 'Neck': 'Moc...","[{'rating': '5', 'review_text': 'Super fit but...","[https://assets.myntassets.com/h_720,q_90,w_54...",men-sweatshirts,https://www.myntra.com/sweatshirts/quechua+by+...
3,H&M,Loose Fit Zip-Through Hoodie,MRP,MRP₹ 1499,Zip-through hoodie in lightweight sweatshirt f...,"{'Sleeve Length': 'Long Sleeves', 'Neck': 'Hoo...","[{'rating': '5', 'review_text': 'Myntra is the...","[https://assets.myntassets.com/h_720,q_90,w_54...",men-sweatshirts,https://www.myntra.com/sweatshirts/h%26m/hm-lo...
4,Roadster,The Lifestyle Co. Men Mock Collar Sweatshirt,MRP ₹1099,₹604,"Olive green solid sweatshirt, has a mock colla...","{'Sleeve Length': 'Long Sleeves', 'Neck': 'Moc...","[{'rating': '5', 'review_text': 'Roadster is a...","[https://assets.myntassets.com/h_720,q_90,w_54...",men-sweatshirts,https://www.myntra.com/sweatshirts/roadster/th...
...,...,...,...,...,...,...,...,...,...,...
693,SASSAFRAS,Black Front Slit Denim Wrap Midi Skirt,MRP ₹2199,₹923,"Black solid wrap midi skirt, has an front slit...","{'Type': 'Wrap', 'Length': 'Midi', 'Print or P...","[{'rating': '4', 'review_text': 'Very trending...","[https://assets.myntassets.com/h_720,q_90,w_54...",women-shorts-skirts,https://www.myntra.com/skirts/sassafras/sassaf...
694,SASSAFRAS BASICS,Pencil Midi Skirt,MRP ₹1399,₹573,"Beige solid straight skirt, has an elasticated...","{'Type': 'Pencil', 'Length': 'Midi', 'Print or...","[{'rating': '4', 'review_text': 'Very comfy. E...","[https://assets.myntassets.com/h_720,q_90,w_54...",women-shorts-skirts,https://www.myntra.com/skirts/sassafras+basics...
695,Tokyo Talkies,Women Black Solid Regular Shorts,MRP ₹699,₹279,"Black solid mid-rise regular shorts, has 2 poc...","{'Waist Rise': 'Mid-Rise', 'Length': 'Above Kn...","[{'rating': '5', 'review_text': 'I don't know ...","[https://assets.myntassets.com/h_720,q_90,w_54...",women-shorts-skirts,https://www.myntra.com/shorts/tokyo+talkies/to...
696,StyleCast,Straight Maxi Skirts,MRP ₹2449,₹899,"Black solid maxi skirt, has an elasticated wai...","{'Type': 'Straight', 'Length': 'Maxi', 'Print ...",[],"[https://assets.myntassets.com/h_720,q_90,w_54...",women-shorts-skirts,https://www.myntra.com/skirts/stylecast/stylec...


### Feature Engineering 

1. Using prompt engineering to get GPT-4o to generate detailed product descriptions.
2. Combining the product details and specifications to form a consistent final product description.
3. Summarizing user feedback.

In [16]:
# Combine product details and specifications
df['Combined Description'] = df.apply(
    lambda row: f"{row['Product Details']} Specifications: {', '.join([f'{k}: {v}' for k, v in row['Specifications'].items()])}", 
    axis=1
)

# Drop original columns if no longer needed
df = df.drop(columns=['Product Details', 'Specifications'])


In [17]:
df["Reviews"][0]

[{'rating': '5',
  'review_text': "That's exactly what I needed it, sweatshirt with superb soft thermal fabric inside & perfect for Delhi winters, love that chocolate brown colour, same as shown in their images. I have gone with XL size which is quite a relaxing fit for me as my height is 6feet and I don't mind wearing it even while sleeping..but if anyone is looking for exact fit, go for one size smaller then your normal size.... Simple but quite advisable will surely be recommending this and till then I'm buying one more for sure....thanks a lot H&M ....Last but not least I would request myntra to work on product description as none of your product description states that the fabric is suitable for summer or winter, plz do the needful as it may help us a lot and we can ignore return/exchange atleast..!! ??",
  'reviewer_name': 'Manish Sharma',
  'review_date': '6 Nov 2021'}]

To make the reviews section more readable, we can extract the rating from each review and calculate the average. For the review text, we can create a list of lists and then summarize it. Using a list of lists helps maintain the integrity of individual reviews, preventing issues like combining separate reviews into a single, incoherent text.

In [18]:
def summarize_reviews(reviews):
    if not reviews:
        return {"average_rating": None, "summary": []}
    
    # Calculate average rating
    ratings = [int(review['rating']) for review in reviews]
    average_rating = sum(ratings) / len(ratings)
    
    # Summarize review text
    review_texts = [[review['review_text']] for review in reviews]
    
    return {"average_rating": average_rating, "summary": review_texts}

# Apply the function to the 'Reviews' column
df['Review Summary'] = df['Reviews'].apply(summarize_reviews)

# Split the summary into separate columns
df['Average Rating'] = df['Review Summary'].apply(lambda x: x['average_rating'])
df['Review Text Summary'] = df['Review Summary'].apply(lambda x: x['summary'])
df = df.drop(columns=['Reviews', 'Review Summary'])


In [19]:
df["Review Text Summary"][0]

[["That's exactly what I needed it, sweatshirt with superb soft thermal fabric inside & perfect for Delhi winters, love that chocolate brown colour, same as shown in their images. I have gone with XL size which is quite a relaxing fit for me as my height is 6feet and I don't mind wearing it even while sleeping..but if anyone is looking for exact fit, go for one size smaller then your normal size.... Simple but quite advisable will surely be recommending this and till then I'm buying one more for sure....thanks a lot H&M ....Last but not least I would request myntra to work on product description as none of your product description states that the fabric is suitable for summer or winter, plz do the needful as it may help us a lot and we can ignore return/exchange atleast..!! ??"]]

In [20]:
df.iloc[0]

Product Title                                                             H&M
Product Name                                           Relaxed Fit Sweatshirt
Product Price                                                             MRP
Product Discounted Price                                             MRP₹ 799
Image URLs                  [https://assets.myntassets.com/h_720,q_90,w_54...
Product Type                                                  men-sweatshirts
Buy Link                    https://www.myntra.com/sweatshirts/h%26m/hm-re...
Combined Description        Top in sweatshirt fabric made from a cotton bl...
Average Rating                                                            5.0
Review Text Summary         [[That's exactly what I needed it, sweatshirt ...
Name: 0, dtype: object

In [56]:
import os
import requests
import base64

# Configuration
IMAGE_PATH = "/Users/suhaniagarwal/ecommerce-scrapper/__node_modules_reactnative_libraries_logbox_ui_logboximages_alerttriangle.png"
encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
# Configuration
API_KEY = "f85804b527a943f197ee55b7df623528"

headers = {
    "Content-Type": "application/json",
    "api-key": API_KEY,
}

# Function to get the first image URL from the DataFrame
def get_first_image_url(image_urls):
    return image_urls[0] if image_urls else ""

# Function to generate description for a product
def generate_description(combined_description, first_image_url):
    payload = {
        "messages": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": "You are an AI stylist that helps in generating a description for the product image. You will be given product specifications and description, so utilize it to generate an informative yet short description. You can capture things like what body type it will look nice on."
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": combined_description
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": first_image_url
                        }
                    }
                ]
            }
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 300
    }
    ENDPOINT = "https://embedding-model12.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

    # ENDPOINT = "https://gpt-4-newmms.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2023-03-15-preview"

    try:
        response = requests.post(ENDPOINT, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
        a = response.json()
        return a['choices'][0]['message']['content']
    except requests.RequestException as e:
        print(f"Failed to make the request for description. Error: {e}")
        return None

# Loop through each product in the DataFrame
# for index, row in df.iterrows():
#     # Generate description for the current product
#     description = generate_description(row['Combined Description'], get_first_image_url(row['Image URLs']))
    
#     # Store the generated description in the DataFrame
#     df.at[index, 'Generated Description'] = description

#     # Save every 10 requests
#     if (index + 1) % 2 == 0:
#         df.to_csv("extracted_product_data_with_descriptions.csv", index=False)  # Save up to current index
#         print(f"Saved after processing {index + 1} products.")

# # Final save after all requests
# df.to_csv("extracted_product_data_with_descriptions.csv", index=False)
# print("Final save completed.")


# Loop through each product in the DataFrame starting from index 460
for index in range(460, len(df)):
    row = df.iloc[index]  # Get the current row

    # Generate description for the current product
    description = generate_description(row['Combined Description'], get_first_image_url(row['Image URLs']))
    
    # Store the generated description in the DataFrame
    df.at[index, 'Generated Description'] = description

    # Save every 10 requests
    if (index + 1) % 10 == 0:  # Changed to save every 10 requests
        df.to_csv("extracted_product_data_with_descriptions.csv", index=False)  # Save up to current index
        print(f"Saved after processing {index + 1} products.")

# Final save after all requests
df.to_csv("extracted_product_data_with_descriptions.csv", index=False)
print("Final save completed.")

KeyError: 'Combined Description'

In [27]:
df

,Product Title,Product Name,Product Price,Product Discounted Price,Image URLs,Product Type,Buy Link,Combined Description,Average Rating,Review Text Summary,Generated Description
0,H&M,Relaxed Fit Sweatshirt,MRP,MRP₹ 799,"[https://assets.myntassets.com/h_720,q_90,w_54...",men-sweatshirts,https://www.myntra.com/sweatshirts/h%26m/hm-re...,Top in sweatshirt fabric made from a cotton bl...,5.000000,"[[That's exactly what I needed it, sweatshirt ...",This relaxed-fit pullover sweatshirt is crafte...
1,Roadster,Men Charcoal Solid Hooded Sweatshirt,MRP ₹999,₹599,"[https://assets.myntassets.com/h_720,q_90,w_54...",men-sweatshirts,https://www.myntra.com/sweatshirts/roadster/ro...,"Charcoal solid sweatshirt, has a hood, long sl...",5.000000,[[I have been using the sweatshirt from couple...,This charcoal solid sweatshirt is a versatile ...
2,Quechua By Decathlon,Men Orange Solid Hiking and Trekking Sweatshirt,MRP ₹899,₹699,"[https://assets.myntassets.com/h_720,q_90,w_54...",men-sweatshirts,https://www.myntra.com/sweatshirts/quechua+by+...,Orange sweathirt for men\nPattern: solid\nRegu...,4.666667,[[Super fit but do get your measurements right...,This men's orange sweatshirt features a solid ...
3,H&M,Loose Fit Zip-Through Hoodie,MRP,MRP₹ 1499,"[https://assets.myntassets.com/h_720,q_90,w_54...",men-sweatshirts,https://www.myntra.com/sweatshirts/h%26m/hm-lo...,Zip-through hoodie in lightweight sweatshirt f...,5.000000,[[Myntra is the best shopping app I have ever ...,This zip-through hoodie is crafted from lightw...
4,Roadster,The Lifestyle Co. Men Mock Collar Sweatshirt,MRP ₹1099,₹604,"[https://assets.myntassets.com/h_720,q_90,w_54...",men-sweatshirts,https://www.myntra.com/sweatshirts/roadster/th...,"Olive green solid sweatshirt, has a mock colla...",5.000000,[[Roadster is all wise best in quality and lit...,This olive green solid sweatshirt features a m...
...,...,...,...,...,...,...,...,...,...,...,...
693,SASSAFRAS,Black Front Slit Denim Wrap Midi Skirt,MRP ₹2199,₹923,"[https://assets.myntassets.com/h_720,q_90,w_54...",women-shorts-skirts,https://www.myntra.com/skirts/sassafras/sassaf...,"Black solid wrap midi skirt, has an front slit...",4.000000,"[[Very trending skirt,must have wardrobe staple]]",This black solid wrap midi skirt is a chic add...
694,SASSAFRAS BASICS,Pencil Midi Skirt,MRP ₹1399,₹573,"[https://assets.myntassets.com/h_720,q_90,w_54...",women-shorts-skirts,https://www.myntra.com/skirts/sassafras+basics...,"Beige solid straight skirt, has an elasticated...",4.000000,[[Very comfy. Easy to wear. Exact show in pic....,This elegant beige solid straight skirt is a v...
695,Tokyo Talkies,Women Black Solid Regular Shorts,MRP ₹699,₹279,"[https://assets.myntassets.com/h_720,q_90,w_54...",women-shorts-skirts,https://www.myntra.com/shorts/tokyo+talkies/to...,"Black solid mid-rise regular shorts, has 2 poc...",5.000000,[[I don't know if the actual color is visible ...,These black solid mid-rise shorts offer a perf...
696,StyleCast,Straight Maxi Skirts,MRP ₹2449,₹899,"[https://assets.myntassets.com/h_720,q_90,w_54...",women-shorts-skirts,https://www.myntra.com/skirts/stylecast/stylec...,"Black solid maxi skirt, has an elasticated wai...",NaN,[],This black solid maxi skirt is a versatile add...


In [28]:
df = df.drop(columns=['Combined Description'])
df.iloc[0]

Product Title                                                             H&M
Product Name                                           Relaxed Fit Sweatshirt
Product Price                                                             MRP
Product Discounted Price                                             MRP₹ 799
Image URLs                  [https://assets.myntassets.com/h_720,q_90,w_54...
Product Type                                                  men-sweatshirts
Buy Link                    https://www.myntra.com/sweatshirts/h%26m/hm-re...
Average Rating                                                            5.0
Review Text Summary         [[That's exactly what I needed it, sweatshirt ...
Generated Description       This relaxed-fit pullover sweatshirt is crafte...
Name: 0, dtype: object

In [29]:
import os
from openai import AzureOpenAI
import pandas as pd

# Configuration
API_KEY = "f85804b527a943f197ee55b7df623528"
API_VERSION = "2024-02-15-preview"
AZURE_ENDPOINT = "https://embedding-model12.openai.azure.com"

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    api_key=API_KEY,
    api_version=API_VERSION,
    azure_endpoint=AZURE_ENDPOINT
)

def summarize_reviews(review_text_summary, model="gpt-4o"):
    payload = {
        "messages": [
            {
                "role": "system",
                "content": "You are an AI assistant that summarizes the reviews written by users for a product in less than 50 words"
            },
            {
                "role": "user",
                "content": review_text_summary
            }
        ],
        "temperature": 0.62,
        "top_p": 0.76,
        "max_tokens": 213
    }
    
    try:
        response = client.chat.completions.create(
            model=model,
            messages=payload["messages"]
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Failed to get response from OpenAI. Error: {e}")
        return None


for index in range(len(df)):
    # Summarize reviews for the current product
    review_summary = summarize_reviews(str(df['Review Text Summary'].iloc[index]))
    
    # Store the generated summary in the DataFrame
    df.at[index, 'Review Summary'] = review_summary

    # Save every 10 requests
    if (index + 1) % 5 == 0:  # Save every 10 products
        df.to_csv("extracted_product_data_with_review_summaries.csv", index=False)  # Save up to current index
        print(f"Saved after processing {index + 1} products.")

# Final save after all requests
df.to_csv("extracted_product_data_with_review_summaries.csv", index=False)
print("Final save completed.")

Saved after processing 5 products.
Saved after processing 10 products.
Saved after processing 15 products.
Saved after processing 20 products.
Saved after processing 25 products.
Saved after processing 30 products.
Saved after processing 35 products.
Saved after processing 40 products.
Saved after processing 45 products.
Saved after processing 50 products.
Saved after processing 55 products.
Saved after processing 60 products.
Saved after processing 65 products.
Saved after processing 70 products.
Saved after processing 75 products.
Saved after processing 80 products.
Saved after processing 85 products.
Saved after processing 90 products.
Saved after processing 95 products.
Saved after processing 100 products.
Saved after processing 105 products.
Saved after processing 110 products.
Saved after processing 115 products.
Saved after processing 120 products.
Saved after processing 125 products.
Saved after processing 130 products.
Saved after processing 135 products.
Saved after processin

In [30]:
df = df.drop(columns=['Review Text Summary'])


In [32]:
df["Review Summary"][1]

'Users praise the sweatshirt for its great fit, high-quality fabric, and comfort. They find it awesome for the price, with a thick, comfy material ideal for winters. Highly recommended!'

In [33]:
df.iloc[0]

Product Title                                                             H&M
Product Name                                           Relaxed Fit Sweatshirt
Product Price                                                             MRP
Product Discounted Price                                             MRP₹ 799
Image URLs                  [https://assets.myntassets.com/h_720,q_90,w_54...
Product Type                                                  men-sweatshirts
Buy Link                    https://www.myntra.com/sweatshirts/h%26m/hm-re...
Average Rating                                                            5.0
Generated Description       This relaxed-fit pullover sweatshirt is crafte...
Review Summary              The sweatshirt features superb soft thermal fa...
Name: 0, dtype: object

In [34]:
df.to_csv("output_summaries.csv", index=False)

### Translation - using API

1. using sarvam api to translate the product description in hindi,kannada[doing 2 because of resource contraints]

In [35]:
import requests

# Function to translate text using Sarvam API
def translate_text(text, target_language_code):
    url = "https://api.sarvam.ai/translate"
    payload = {
        "input": text,
        "source_language_code": "en-IN",
        "target_language_code": target_language_code,
        "speaker_gender": "Male",
        "mode": "formal",
        "model": "mayura:v1",
        "enable_preprocessing": True
    }
    headers = {
        "Content-Type": "application/json",
        "API-Subscription-Key": "2172d237-dbf0-43f4-b8d5-941fb2ea3b78"
    }
    response = requests.request("POST", url, json=payload, headers=headers)
    return response.json().get("translated_text")

# Translate columns to Hindi (hi) and Kannada (kn)
df['Product Title Hindi'] = df['Product Title'].apply(lambda x: translate_text(x, 'hi-IN'))
df['Product Name Hindi'] = df['Product Name'].apply(lambda x: translate_text(x, 'hi-IN'))
df['Generated Description Hindi'] = df['Generated Description'].apply(lambda x: translate_text(x, 'hi-IN'))
df['Review Summary Hindi'] = df['Review Summary'].apply(lambda x: translate_text(x, 'hi-IN'))

# df['Product Title Kannada'] = df['Product Title'].apply(lambda x: translate_text(x, 'kn-IN'))
# df['Product Name Kannada'] = df['Product Name'].apply(lambda x: translate_text(x, 'kn-IN'))
# df['Generated Description Kannada'] = df['Generated Description'].apply(lambda x: translate_text(x, 'kn-IN'))
# df['Review Summary Kannada'] = df['Review Summary'].apply(lambda x: translate_text(x, 'kn-IN'))

ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [38]:
import requests
import pandas as pd

# Function to translate text using Sarvam API
def translate_text(text, target_language_code):
    url = "https://api.sarvam.ai/translate"
    payload = {
        "input": text,
        "source_language_code": "en-IN",
        "target_language_code": target_language_code,
        "speaker_gender": "Male",
        "mode": "formal",
        "model": "mayura:v1",
        "enable_preprocessing": True
    }
    headers = {
        "Content-Type": "application/json",
        "API-Subscription-Key": "fed396c4-aecc-4f85-a062-7d17497d7335"
    }
    response = requests.request("POST", url, json=payload, headers=headers)
    return response.json().get("translated_text")

# Function to translate a column in the DataFrame
def translate_column(column_name, target_language_code):
    for index in range(len(df)):
        # Translate the text for the current row
        translated_text = translate_text(str(df[column_name].iloc[index]), target_language_code)
        
        # Store the translated text in a new column
        df.at[index, f'{column_name} {target_language_code}'] = translated_text

        # Save every 10 requests
        if (index + 1) % 5 == 0:  # Save every 10 products
            df.to_csv("extracted_product_data_translated.csv", index=False)  # Save up to current index
            print(f"Saved after processing {index + 1} products.")

# Load your DataFrame (make sure to adjust the path as necessary)

# Translate columns to Hindi (hi) and Kannada (kn)
translate_column('Product Title', 'hi-IN')
translate_column('Product Name', 'hi-IN')
translate_column('Generated Description', 'hi-IN')
translate_column('Review Summary', 'hi-IN')

# Uncomment the following lines to translate to Kannada
# translate_column('Product Title', 'kn-IN')
# translate_column('Product Name', 'kn-IN')
# translate_column('Generated Description', 'kn-IN')
# translate_column('Review Summary', 'kn-IN')

# Final save after all translations
df.to_csv("extracted_product_data_translated.csv", index=False)
print("Final save completed.")

Saved after processing 5 products.
Saved after processing 10 products.
Saved after processing 15 products.
Saved after processing 20 products.
Saved after processing 25 products.
Saved after processing 30 products.
Saved after processing 35 products.
Saved after processing 40 products.
Saved after processing 45 products.
Saved after processing 50 products.
Saved after processing 55 products.
Saved after processing 60 products.
Saved after processing 65 products.
Saved after processing 70 products.
Saved after processing 75 products.
Saved after processing 80 products.
Saved after processing 85 products.
Saved after processing 90 products.
Saved after processing 95 products.
Saved after processing 100 products.
Saved after processing 105 products.
Saved after processing 110 products.
Saved after processing 115 products.
Saved after processing 120 products.
Saved after processing 125 products.
Saved after processing 130 products.
Saved after processing 135 products.
Saved after processin

In [51]:
df=pd.read_csv("extracted_product_data_translated copy 2.csv")
df

,Product Title,Product Name,Product Price,Product Discounted Price,Image URLs,Product Type,Buy Link,Average Rating,Generated Description,Review Summary,Product Title hi-IN,Product Name hi-IN,Generated Description hi-IN,Review Summary hi-IN
0,H&M,Relaxed Fit Sweatshirt,MRP,MRP₹ 799,"['https://assets.myntassets.com/h_720,q_90,w_5...",men-sweatshirts,https://www.myntra.com/sweatshirts/h%26m/hm-re...,5.000000,This relaxed-fit pullover sweatshirt is crafte...,The sweatshirt features superb soft thermal fa...,H&M,रिलैक्स्ड फिट स्वेटशर्ट,यह आरामदायक पुलओवर स्वेटशर्ट एक कोमल सूती मिश्...,"स्वेटशर्ट बहुत नरम, गर्म कपड़े से बना है, जो द..."
1,Roadster,Men Charcoal Solid Hooded Sweatshirt,MRP ₹999,₹599,"['https://assets.myntassets.com/h_720,q_90,w_5...",men-sweatshirts,https://www.myntra.com/sweatshirts/roadster/ro...,5.000000,This charcoal solid sweatshirt is a versatile ...,"Users praise the sweatshirt for its great fit,...",रोडस्टर,पुरुषों का चारकोल सॉलिड हुडेड स्वेटशर्ट,यह चारकोल रंग का स्वेटशर्ट आपके कपड़ों में होना...,लोग वास्तव में सीवनदार कमीज को पसंद करते हैं क...
2,Quechua By Decathlon,Men Orange Solid Hiking and Trekking Sweatshirt,MRP ₹899,₹699,"['https://assets.myntassets.com/h_720,q_90,w_5...",men-sweatshirts,https://www.myntra.com/sweatshirts/quechua+by+...,4.666667,This men's orange sweatshirt features a solid ...,Users praise the product for its great fit (wh...,डेकाथलॉन द्वारा क्वेचुआ,यह पुरुषों का नारंगी ठोस हाइकिंग और ट्रेकिंग स...,पुरुषों के लिए इस नारंगी स्वेटशर्ट में एक साधा...,कई उपयोगकर्ता उत्पाद के उत्कृष्ट फिट से खुश है...
3,H&M,Loose Fit Zip-Through Hoodie,MRP,MRP₹ 1499,"['https://assets.myntassets.com/h_720,q_90,w_5...",men-sweatshirts,https://www.myntra.com/sweatshirts/h%26m/hm-lo...,5.000000,This zip-through hoodie is crafted from lightw...,Myntra is praised for its user-friendly interf...,H&M,ढीले फिट जिप-थ्रू हुडी,"यह जिप-थ्रू हुडी एक हल्की, नरम स्वेटशर्ट सामग्...","मिंत्रा अपनी उपयोग में आसान वेबसाइट और ऐप, विभ..."
4,Roadster,The Lifestyle Co. Men Mock Collar Sweatshirt,MRP ₹1099,₹604,"['https://assets.myntassets.com/h_720,q_90,w_5...",men-sweatshirts,https://www.myntra.com/sweatshirts/roadster/th...,5.000000,This olive green solid sweatshirt features a m...,Users praise the quality and softness of the R...,रोडस्टर,NaN,यह जैतूनी हरी स्वेटशर्ट एक ठोस रंग है और इसमें...,लोग वास्तव में रोडस्टर उत्पाद को पसंद करते हैं...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,SASSAFRAS,Black Front Slit Denim Wrap Midi Skirt,MRP ₹2199,₹923,"['https://assets.myntassets.com/h_720,q_90,w_5...",women-shorts-skirts,https://www.myntra.com/skirts/sassafras/sassaf...,4.000000,This black solid wrap midi skirt is a chic add...,"A highly recommended, trendy skirt that's cons...",NaN,यह एक स्कर्ट का वर्णन है। स्कर्ट डेनिम से बनी ...,यह काली ठोस रैप मिडी स्कर्ट एक स्टाइलिश टुकड़ा...,NaN
694,SASSAFRAS BASICS,Pencil Midi Skirt,MRP ₹1399,₹573,"['https://assets.myntassets.com/h_720,q_90,w_5...",women-shorts-skirts,https://www.myntra.com/skirts/sassafras+basics...,4.000000,This elegant beige solid straight skirt is a v...,Comfortable and easy to wear with a ribbed tex...,NaN,पेंसिल मिडी स्कर्ट,यह बेज स्कर्ट आपके कपड़ों में होना एक अच्छा विच...,NaN
695,Tokyo Talkies,Women Black Solid Regular Shorts,MRP ₹699,₹279,"['https://assets.myntassets.com/h_720,q_90,w_5...",women-shorts-skirts,https://www.myntra.com/shorts/tokyo+talkies/to...,5.000000,These black solid mid-rise shorts offer a perf...,Users love these shorts for their beautiful co...,टोक्यो टॉकीज़,महिलाओं के लिए काले रंग के नियमित शॉर्ट्स,ये काले ठोस मध्य-ऊंची शॉर्ट्स आराम और फैशन दोन...,NaN
696,StyleCast,Straight Maxi Skirts,MRP ₹2449,₹899,"['https://assets.myntassets.com/h_720,q_90,w_5...",women-shorts-skirts,https://www.myntra.com/skirts/stylecast/stylec...,NaN,This black solid maxi skirt is a versatile add...,There are no reviews available for this product.,NaN,सीधी मैक्सी स्कर्ट,यह काली मैक्सी स्कर्ट आपके अनौपचारिक कपड़ों के...,NaN


In [52]:
# Check for missing translations in the specified columns
missing_rows = {}

# List of columns to check for missing translations
columns_to_check = [
    'Product Title hi-IN',
    'Product Name hi-IN',
    'Generated Description hi-IN',
    'Review Summary hi-IN'
]

# Iterate through each column and find missing rows
for column in columns_to_check:
    missing_indices = df[df[column].isna()].index.tolist()
    if missing_indices:
        missing_rows[column] = missing_indices

# Print the missing rows for each column and their counts
for column, indices in missing_rows.items():
    print(f"Missing translations in '{column}': {indices} (Count: {len(indices)})")

Missing translations in 'Product Title hi-IN': [60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 215, 216, 217, 227, 233, 239, 243, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 31

In [31]:
df.to_csv("extracted_product_data_translated.csv", index=False)

### Translation - using instruction


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextGenerationPipeline

# initialize a HF pipeline for easy use
model = AutoModelForCausalLM.from_pretrained("sarvamai/sarvam-2b-v0.5")
tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-2b-v0.5")
tokenizer.pad_token_id = tokenizer.eos_token_id
pipe = TextGenerationPipeline(model=model, tokenizer=tokenizer, device="cuda", torch_dtype="bfloat16", return_full_text=False)

In [ ]:
gen_kwargs = {
  "temperature": 0.01, # we are introducing a small amount of stocasticity in the model generation process
  "repetition_penalty": 1.2, # discourage the model from repeating recently generated tokens
  "max_new_tokens": 256, # maximum number of tokens to generate
  "stop_strings": ["</s>", "\n\n"], # if these tokens are seen, stop the generation
  "tokenizer": tokenizer, # tokenizer of the model
} # works best with these defaults

# a simple wrapper to get text completions
def gen(prompt):
  # having a trailing space hurts the model generation; let us strip them
  prompt = prompt.rstrip()
  output = pipe(prompt, **gen_kwargs)
  print(output[0]["generated_text"])

In [ ]:
prompt = """English: The main motive of the Manhattan Project was to develop a nuclear bomb
Hindi: मैनहट्टन परियोजना का मुख्य उद्देश्य परमाणु बम विकसित करना था

English: It was done during World War II and was a major factor during the last stages of the war.
Hindi: यह द्वितीय विश्व युद्ध के दौरान किया गया था और युद्ध के अंतिम चरणों के दौरान एक प्रमुख कारक था।

English: The current population of earth is about to exceed 8 billion, but experts say that this is not a cause for alarm.
Hindi: पृथ्वी की वर्तमान जनसंख्या 8 अरब से अधिक होने वाली है, लेकिन विशेषज्ञों का कहना है कि यह चिंता का कारण नहीं है।

English: The first horcrux that Harry Potter destroyed was Tom Riddle's diary.
Hindi:"""

gen(prompt)